# Prep CESM-DPLE experiment data for use in DPLE processing notebook
### • eliminates depth dimension

In [1]:
import os
from glob import glob

import matplotlib.colors as colors

import numpy as np
import xarray as xr
import pop_tools
import utils

import warnings
warnings.filterwarnings('ignore')

## Spin up a dask cluster

In [2]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='50GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=50GB',
        project='NCGD0011',
        walltime='24:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

In [3]:
cluster, client = get_ClusterClient()
cluster.scale(12) 

In [4]:
client

Client Scheduler: tcp://10.12.206.57:39556 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/39402/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
grid = pop_tools.get_grid('POP_gx1v6')
KMT = grid.KMT

In [6]:
def field_at_bottom(da):
    """return a field indexed at the model's bottom layer"""

    tmp_bot = xr.DataArray(np.ones(da[:, 0, :, :].shape) * np.nan, 
                           dims=tuple(da.dims[i] for i in [0, 2, 3]),
                           coords={c: da.coords[c] for c in ['time']},
                          )

    assert KMT.shape == da.shape[-2:]
    
    for j in range(len(da.nlat)):
        for i in range(len(da.nlon)):
            if KMT[j, i] > 0:
                k = int(KMT[j, i] - 1)
                tmp_bot.values[:, j, i] = da[:, k, j, i]
                
    return tmp_bot

## cycle through start years (1954 - 2017; n=64) and ensemble members (n=40) and do preprocessing

In [7]:
#user set/ cycle vars
var = 'TEMP'

for styear in np.arange(2008,2018,1):
    
    for member in np.arange(1,41,1):
    #for member in np.arange(27,41,1):
        print('start year=', styear, 'member=', member)
        
        #first get this particular file
        ds = xr.open_dataset(
        utils.file_in(var,styear,member), 
        decode_coords=False, 
        decode_times=True,
        chunks={'time': 12}).persist()

        #tidy up the dataset
        coords = {'x':'TLONG','y':'TLAT'}
        keepthese=['z_t','z_t_150m','time_bound','TAREA','REGION_MASK','time','UAREA'] + [var]
        keep_vars = keepthese +list(coords.values())+['dz','KMT']
        ds = ds.drop([v for v in ds.variables if v not in keep_vars])

        #prepare output file -- it needs certain variables add to work with the DPLE code
        USER = os.environ['USER']
        dout = f'/glade/scratch/{USER}/DPLE-tmpfiles/'
        os.makedirs(dout, exist_ok=True)
        file_out = utils.file_out(dout,var,styear,member)
        dso = ds[['TLONG', 'TLAT', 'KMT', 'TAREA','time','time_bound','dz']] 

        #get the KMT (even though this could be )
        template = ds[var][:, 0, :, :].drop('z_t') 
        dso[f'{var}_bottom'] = xr.map_blocks(
        field_at_bottom, ds[var],
        template=template).compute()

        #fix attributes
        dso[f'{var}_bottom'].attrs['coordinates'] = "TLONG TLAT time"

        #write out the file
        dso.to_netcdf(file_out, mode='w')

start year= 2008 member= 1
start year= 2008 member= 2
start year= 2008 member= 3
start year= 2008 member= 4
start year= 2008 member= 5


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-a47197e1fb9cdea701dfa4be759d3c30HT-48675b4fc1d1cdf5501b8885ca934774', 0, 0)"


start year= 2008 member= 6
start year= 2008 member= 7
start year= 2008 member= 8
start year= 2008 member= 9


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-1478fdd15c48f9c7a27c3b934b58feafDYU-305ae1079f460b04cc38ec9db73feb5e', 0, 0)"


start year= 2008 member= 10
start year= 2008 member= 11
start year= 2008 member= 12
start year= 2008 member= 13
start year= 2008 member= 14
start year= 2008 member= 15
start year= 2008 member= 16
start year= 2008 member= 17
start year= 2008 member= 18
start year= 2008 member= 19
start year= 2008 member= 20
start year= 2008 member= 21
start year= 2008 member= 22
start year= 2008 member= 23
start year= 2008 member= 24
start year= 2008 member= 25
start year= 2008 member= 26
start year= 2008 member= 27
start year= 2008 member= 28
start year= 2008 member= 29
start year= 2008 member= 30
start year= 2008 member= 31
start year= 2008 member= 32
start year= 2008 member= 33
start year= 2008 member= 34
start year= 2008 member= 35
start year= 2008 member= 36
start year= 2008 member= 37
start year= 2008 member= 38
start year= 2008 member= 39
start year= 2008 member= 40
start year= 2009 member= 1
start year= 2009 member= 2


...

start year= 2017 member= 37


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-7c7b0d81b79868dc8475b2d7981211d1DYT-3d0bb4edb5116cb4d2af9dd34fae6d88', 0, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", lin

start year= 2017 member= 38


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-4a65aa2fca5c9d0847c7c354fb250fecANGLET-1503a2080a6659bed28ac5795a7b177f', 0, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", 

start year= 2017 member= 39
start year= 2017 member= 40


In [11]:
#cluster.close()